In [ ]:
import numpy as np
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi
from collections import defaultdict

In [ ]:
N = 128
L = 10
sigma02 = (L**2)/N
x = torch.rand((N,2))*L
x = x.cuda().requires_grad_(True)
sigma2 = torch.rand(N)*sigma02 + sigma02
sigma2 = sigma2.cuda().requires_grad_(True)
gamma_b = 1
gamma_s = 1.0

In [ ]:
lr = 0.001
for step in range(1000):
    
    with torch.no_grad():
        mask = torch.isfinite(sigma2)
        x = x[mask].requires_grad_(True)
        sigma2 = sigma2[mask].requires_grad_(True)
    with torch.no_grad():
        x_cpu = x.cpu().numpy()
        vor = Voronoi(x_cpu, incremental=False)
        edgelist = torch.LongTensor(vor.ridge_points).t().cuda()
        plt.scatter(x_cpu[:,0], x_cpu[:,1])
        plt.show()
    dx_interacting = torch.diff(x[edgelist],dim=0).squeeze(0)
    sigma2_interacting = sigma2[edgelist]
    sum_of_sigma2 = torch.sum(sigma2_interacting, dim=0)
    prod_of_sigma2 = torch.prod(sigma2_interacting, dim=0)
    dx2_over_ss2 = (torch.sum(dx_interacting**2.0, dim=1))/sum_of_sigma2
    overlaps = (prod_of_sigma2/sum_of_sigma2)*torch.exp(-0.5*dx2_over_ss2)
    bulk_and_surface_terms = gamma_b + gamma_s*( 2 - dx2_over_ss2 )/sum_of_sigma2
    energy = (overlaps*bulk_and_surface_terms).sum()
    energy.backward()
    with torch.no_grad():
        x = x - lr*x.grad
        sigma2 = sigma2 -  lr*sigma2.grad

In [ ]:
with torch.no_grad():
    x_cpu = x.cpu().numpy()
    sigma2_cpu = sigma2.cpu().numpy()
    plt.scatter(x_cpu[:,0], x_cpu[:,1])